In [5]:
import kaggle
from sklearn import *
import pandas as pd
import seaborn as sns
import kagglehub
from kaggle.api.kaggle_api_extended import KaggleApi

In [6]:
# Initiliaze API
api = KaggleApi()
api.authenticate()
api.model_list_cli()

# Donwload Dataset via API unzipped
api.dataset_download_files(dataset="prathamtripathi/drug-classification", unzip=True)


Next Page Token = CfDJ8CXYA35d3CRDujxBNSrCTMu8GK_IK7YyR08nbJLcrG1mCuS1UGQUSy6dyK8qWGQacrVrHiitCEswFJnOiiWQRfM
    id  ref                                         title                             subtitle                                                                                                                                                                                                              author                  
------  ------------------------------------------  --------------------------------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------  
121027  metaresearch/llama-3.2                      Llama 3.2                         The Meta Llama 3.2 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction-tuned generative models in 1B and